# WCA Results - Partial Senior Rankings

Created by Michael George (AKA Logiqx)

Link: https://www.speedsolving.com/forum/showthread.php?54128-How-fast-are-the-over-40-s-in-competitions

In [1]:
from EventsLib import *

## Read Partial Results from CSV

Read event data from CSV into memory, prior to processing

In [2]:
import os, csv

class PartialResults:
    
    def __init__(self, ageCategory):
        """Initialisise the partial results"""
        
        self.event = None
        self.persons = {}
        self.ids = []
        self.results = {}
        self.ageCategory = ageCategory


    def readPersons(self, basename):
        """Read seniors from CSV into memory"""
        
        self.persons = {}

        # Read rows using the CSV reader
        fn = os.path.join('..', 'data', 'public', 'ready', basename + '.csv')
        with open(fn, 'r') as f:
            csvReader = csv.reader(f)
            
            # Process each row individually
            for person in csvReader:
                personAge = person[4]
                wcaId = person[0]

                if int(personAge) >= self.ageCategory and wcaId not in self.persons:
                    self.persons[wcaId] = person[1:]
                    self.ids.append(wcaId)

                
    def listPersons(self):
        """List seniors from memory"""
        
        html = '<details id="names">\n'
        html += '  <summary>%s</summary>\n' % 'Known Over %ds' % self.ageCategory
        html += '  <table>\n'
        html += '    <tr>'
        for field in ['Person', 'Speedsolving.com']:
            html += '<td><b>%s</b></td>' % field
        html += '</tr>\n'
            
        for id in self.ids:
            person = self.persons[id]
            
            name = person[0]
            country = person[1]
            age = ', %d+' % int(person[3]) if int(person[3]) > self.ageCategory else ''
            link = '<a href="https://www.worldcubeassociation.org/persons/%s">%s</a>' % (id, name)

            html += '    <tr>'
            html += '<td>%s, %s%s</td>' % (link, country, age)
            html += '<td>%s</td>' % person[2]
            html += '</tr>\n'

        html += '  </table>\n'
        html += '</details>\n\n'
        
        return html
            
        
    def readResults(self, basename, event):
        """Read event results from CSV into memory"""
        
        self.event = event
        self.results = []
        
        if event[0] == '333fm' and 'single' in basename:
            self.fmSingle = True
        else:
            self.fmSingle = False

        # Read rows using the CSV reader
        fn = os.path.join('..', 'data', 'public', 'ready', basename, event[0] + '.csv')
        with open(fn, 'r') as f:
            csvReader = csv.reader(f)
            
            # Process each row individually
            persons = {}
            for inputRow in csvReader:             
                personAge = inputRow[2]
                wcaId = inputRow[0]

                if int(personAge) >= self.ageCategory and wcaId not in persons:
                    self.results.append(inputRow)
                    persons[wcaId] = True
                

    def listResults(self, section):
        """List seniors from memory"""
        
        html = ''
        
        if len(self.results) > 0:
            html = '<details id="%s_%s">\n' % (self.event[0], section)
            html += '  <summary>%s</summary>\n' % self.event[1]
            html += '  <table>\n'
            html += '    <tr>'
            for field in ['Rank', 'Person', 'Result']:
                html += '<td><b>%s</b></td>' % field
            html += '</tr>\n'

            rank = 1
            prevResult = None

            for result in self.results:

                person = self.persons[result[0]]
                name = person[0]
                country = person[1]
                age = ', %d+' % int(result[2]) if int(result[2]) > self.ageCategory else ''

                link = '<a href="https://www.worldcubeassociation.org/persons/%s#%s">%s</a>' % \
                        (result[0], self.event[0], name)

                if self.fmSingle:
                    result = formatResult(self.event, int(result[1]) * 100, showFractions = False)
                else:
                    result = formatResult(self.event, result[1], showFractions = True)

                html += '    <tr>'
                html += '<td style="text-align:center">%s</td>' % (rank if result != prevResult else '')
                html += '<td>%s, %s%s</td>' % (link, country, age)
                html += '<td style="text-align:right">%s</td>' % result
                html += '</tr>\n'

                prevResult = result
                rank += 1

            html += '  </table>\n'
            html += '</details>\n\n'
        
        return html

## Analyse Events

Process the events one-by-one

In [3]:
import datetime

def runReport(ageCategory):
    
    fn = 'Partial_Rankings_%d.md' % ageCategory if ageCategory > 40 else 'Partial_Rankings.md'
    with open(os.path.join('..', 'templates', fn), 'r') as f:
        html = ''.join(f.readlines())

    refreshed = datetime.datetime.now().replace(microsecond=0).isoformat().replace('T', ' ')
    html += 'Last refreshed: ' + refreshed + ' (UTC)\n\n'

    partialResults = PartialResults(ageCategory)

    html += '<h2 id="competitors">%s</h2>\n\n' % 'Official Competitors'
    fn = 'known_senior_details'
    partialResults.readPersons(fn)
    html += partialResults.listPersons()

    ids = partialResults.ids
    ids.sort()
    idsCsv = ','.join(id for id in ids)
    html = html.replace("wcaids=", "wcaids=" + idsCsv)

    html += '<h2 id="averages">%s</h2>\n\n' % 'Official Averages'
    for event in events:
        if event[3] != 'multi':
            fn = 'known_senior_averages'
            partialResults.readResults(fn, event)
            html += partialResults.listResults("avg")

    html += '<h2 id="singles">%s</h2>\n\n' % 'Official Singles'
    for event in events:
        fn = 'known_senior_singles'
        partialResults.readResults(fn, event)
        html += partialResults.listResults("best")

    fn = 'Partial_Rankings_%d.md' % ageCategory if ageCategory > 40 else 'Partial_Rankings.md'
    with open(os.path.join('..', 'docs', fn), 'w') as f:
        f.write(html)

        
runReport(ageCategory = 40)
runReport(ageCategory = 50)
runReport(ageCategory = 60)
runReport(ageCategory = 70)
runReport(ageCategory = 80)

print('Partial Rankings updated!')

Partial Rankings updated!


## All Done!